In [41]:
import numpy as np
from scipy.interpolate import interp1d
from astropy.cosmology import Planck18 as cosmo
import emcee

In [42]:
catalog_data = np.load('../../datasets/sampleDict_FAR_1_in_1_yr.pickle', allow_pickle=True)
selection_data = np.load('../../datasets/injectionDict_FAR_1_in_1.pickle', allow_pickle=True)

In [43]:
z_axis = np.linspace(0,10,100000)
dVdz = cosmo.differential_comoving_volume(z_axis).value/1e9*4*np.pi # Astropy dVcdz is per stradian
dVdz_interp = interp1d(z_axis,dVdz)
def z_distribution_unnormalized(z):
    return dVdz_interp(z)*(1+z)**1.7

z_normalization = np.trapz(z_distribution_unnormalized(z_axis),z_axis)

def z_distribution(z):
    return dVdz_interp(z)*(1+z)**1.7/z_normalization

In [44]:
def truncated_powerlaw(m1,index,mmin=5,mmax=50):
    try:
        output = m1.copy()**index
        index_in = np.where((m1>=mmin)*(m1<=mmax))[0]
        index_out = np.where((m1<mmin)+(m1>mmax))[0]
        normalization = ((mmax)**(1+index)-mmin**(1+index))/(1+index)
        output[index_out] = 1e-30
        output[index_in] = m1[index_in]**index/normalization
    except ZeroDivisionError:
        output = m1.copy()**index
        index_in = np.where((m1>=mmin)*(m1<=mmax))[0]
        index_out = np.where((m1<mmin)+(m1>mmax))[0]
        normalization = np.log(mmax)-np.log(mmin)
        output[index_out] = 1e-30
        output[index_in] = m1[index_in]**index/normalization
    return output

m1_axis = np.linspace(0.01,100,10000)

def M2_distribution(m2,m1,mmin):
    return 3*m2**2/(m1**3-mmin**3)

In [45]:
def likelihood(x,params):
    p_m1 = truncated_powerlaw(x[:,0],params[0],params[1],params[2])
    p_m2 = M2_distribution(x[:,1],x[:,0],mmin=0)
    p_z = z_distribution(x[:,2])
    return p_m1*p_m2*p_z


def selection_bias(likelihood,params):
    m1 = selection_data['m1']
    m2 = selection_data['m2']
    z = selection_data['z']
    p_m1m2z = likelihood(np.array([m1,m2,z]).T,params)
    likelihood_samples = p_m1m2z/selection_data['p_draw_m1m2z']
    return np.sum(likelihood_samples/selection_data['nTrials'])


events_posterior = []
events_prior = []
for event in catalog_data:
    event_data = catalog_data[event]
    if np.median(event_data['m2'])>3:
        posterior = np.stack([event_data['m1'],event_data['m2'],event_data['z']],axis=1)
        prior = event_data['z_prior']/1e9
        events_posterior.append(posterior)
        events_prior.append(prior)

def log_population_likelihood(params):
    selection_bias_ = selection_bias(likelihood,params)
    log_likelihood = 0
    for i in range(len(events_posterior)):
        log_likelihood += np.log(np.mean(likelihood(events_posterior[i],params)/events_prior[i]/selection_bias_))
    
    return log_likelihood

def posterior(params):
    if ((params[0]>1) or (params[0]<0)):
        return -np.inf
    elif (params[1]>1) or (params[1]<0):
        return -np.inf
    elif (params[2]>1) or (params[2]<0):
        return -np.inf
    else:
        params[0] = params[0]*5-10
        params[1] = params[1]*4+3
        params[2] = params[2]*40+60
        output = log_population_likelihood(params)
        if np.isnan(output):
            return -np.inf
        else:
            return output

ndim, nwalkers = 3, 10
ivar = 1. / np.random.rand(ndim)
p0 = np.random.uniform(size=(nwalkers, ndim))


sampler = emcee.EnsembleSampler(nwalkers, ndim, posterior)
result = sampler.run_mcmc(p0, 32768)

In [46]:
chain = sampler.chain

In [47]:
chain = chain[:, 1024:]

In [48]:
chain.shape

(10, 31744, 3)

In [49]:
posterior(chain[-1, -1])

-inf

In [33]:
chain = np.reshape(chain, (-1, 3))

In [34]:
chain.shape

(317440, 3)

In [36]:
posteriors = [posterior(sample) for sample in chain]

posteriors = np.stack(posteriors)

In [40]:
posteriors.argmax()

0

In [ ]:
posterior(chain)